In [18]:
from gu_toolkit import *
from gu_toolkit.SmartParameters import *

# SmartParameter / SmartParameterRegistry — executable sanity checks

This notebook is a **logic + API contract** test suite using plain `assert`s.
Run top-to-bottom. Any failure will raise an `AssertionError` (or the expected exception),
which is the point: it tells you exactly which invariant broke.

Notes:
- These tests assume `SmartParameter.value` **always notifies** callbacks (even if unchanged).
- Some cells are explicitly marked **optional / informational**.
- If you want to emulate the “rerun cell” scenario, re-run the relevant cell(s) after making code changes.

## Imports

Imports used across the tests.
- `gc` is needed for weakref cleanup scenarios.
- `warnings` is used to verify “warn once per notify batch” behavior.

In [19]:
import gc
import warnings

## Defaults, coercion, clamping, unbounded bounds

Verifies:
- Default parameter configuration (type, bounds, default value).
- `value=None` at construction implies `value == default_val`.
- Setting `value` coerces to the declared type (float).
- Values clamp to bounds when bounds are finite.
- `min_val=None` / `max_val=None` mean “unbounded” on that side.

In [20]:
a = Symbol('a')
param_a = SmartParameter(id=a)

assert param_a.id == a
assert param_a.type is float
assert param_a.min_val == -1
assert param_a.max_val == 1
assert param_a.default_val == 0

# Recommended init policy: value defaults to default_val when constructor value=None
assert param_a.value == 0

# Coercion
param_a.value = "0.25"
assert isinstance(param_a.value, float)
assert param_a.value == 0.25

# Clamping
param_a.value = 10
assert param_a.value == 1.0
param_a.value = -10
assert param_a.value == -1.0


# Unbounded above
param_b = SmartParameter(id=Symbol('b'), max_val=None)
param_b.value = 1e6
assert param_b.value == 1e6

# Unbounded below
param_c = SmartParameter(id=Symbol('c'), min_val=None)
param_c.value = -1e6
assert param_c.value == -1e6

## Bounds validation

Verifies that invalid bounds are rejected:
- If `min_val > max_val`, constructor should raise `ValueError`.

In [21]:
try:
    SmartParameter(id=Symbol('bad'), min_val=2, max_val=1)
    raise AssertionError("Expected ValueError for invalid bounds")
except ValueError:
    pass

# Always notify (even if unchanged)
Verifies the “always notify” rule:
- Setting the same value repeatedly still triggers callbacks each time.
- Default `owner_token` passed to callbacks is `None` when using `.value = ...`.
python
Copy code


In [22]:
p = SmartParameter(id=sp.Symbol('x'))
calls = []

def cb(param, **kwargs):
    calls.append((param.value, kwargs.get("owner_token")))

tok = p.register_callback(cb)

p.value = 0.0
p.value = 0.0
p.value = 0.0

assert len(calls) == 3, calls
assert calls[0][1] is None and calls[1][1] is None and calls[2][1] is None


## Token idempotency via scan and removal

Verifies:
- Registering the *same* callback multiple times returns the same token (idempotent).
- Removing a token prevents future calls.
- Removing again is a no-op (should not raise).


In [23]:
p = SmartParameter(id=sp.Symbol('y'))
calls = []

def cb(param, **kwargs):
    calls.append(1)

t1 = p.register_callback(cb)
t2 = p.register_callback(cb)
assert t1 == t2, (t1, t2)

p.value = 0.1
assert len(calls) == 1

p.remove_callback(t1)
p.value = 0.2
assert len(calls) == 1  # no new calls

# removing again should be no-op
p.remove_callback(t1)


## Set_protected excludes exactly the owner token

This tests the “no feedback loop” pattern:
- `set_protected(..., owner_token=t1)` updates the parameter value,
  but does **not** notify the callback associated with `t1`.
- Other callbacks *do* run and see `owner_token=t1`.
- Setting via `.value = ...` notifies all callbacks with `owner_token=None`.


In [24]:
p = SmartParameter(id=sp.Symbol('z'))
log = []

def cb1(param, **kwargs):
    log.append(("cb1", kwargs.get("owner_token")))

def cb2(param, **kwargs):
    log.append(("cb2", kwargs.get("owner_token")))

t1 = p.register_callback(cb1)
t2 = p.register_callback(cb2)

p.set_protected(0.3, owner_token=t1, reason="slider1")

# cb1 excluded, cb2 called
assert ("cb1", t1) not in log
assert ("cb2", t1) in log

log.clear()
p.value = 0.4
assert ("cb1", None) in log and ("cb2", None) in log


## Aggregated errors: continue + warn once + store details

Verifies robustness of notification:
- One bad callback must not prevent others from running.
- All callback failures in a notify batch should be aggregated.
- Recommended: emit **one warning per notify batch** (not one per exception).
- Store detailed error records on the parameter (e.g. `last_callback_errors`),
  including a string traceback for debugging.


In [27]:
p = SmartParameter(id=sp.Symbol('err'))
ran = []

def good(param, **kwargs):
    ran.append("good")

def bad1(param, **kwargs):
    raise RuntimeError("bad1 failed")

def bad2(param, **kwargs):
    raise ValueError("bad2 failed")

p.register_callback(good)
p.register_callback(bad1)
p.register_callback(bad2)

with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    p.value = 0.5

assert "good" in ran

# One warning per notify batch (recommended)
assert len(w) == 1, [str(x.message) for x in w]

errs = getattr(p, "last_callback_errors", None)
assert errs is not None and len(errs) == 2, errs
for e in errs:
    assert hasattr(e, "traceback")
    assert isinstance(e.traceback, str) and len(e.traceback) > 0


##  Weakref cleanup for bound methods (rerun-cell scenario)

This models a common notebook pattern:
- A controller object registers a bound method callback.
- Later, that controller is replaced / goes out of scope (cell rerun).
- The old bound-method callback must not keep the owner alive.
- On notify, dead callbacks should be skipped/cleaned without errors.


In [28]:
p = SmartParameter(id=sp.Symbol('weak'))

class Owner:
    def __init__(self):
        self.count = 0
    def cb(self, param, **kwargs):
        self.count += 1

o = Owner()
tok = p.register_callback(o.cb)

p.value = 0.1
assert o.count == 1

# Drop owner and force GC; callback should become dead
del o
gc.collect()

# Should not crash; dead callback should be cleaned on notify
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    p.value = 0.2

# No callback errors expected from a dead callback; it should be skipped.
assert getattr(p, "last_callback_errors", []) == []


## Weakref requirement for non-weakrefable callbacks (optional / informational)

Some callables (depending on Python build/implementation) may not be weakref-able.
This cell simply reports behavior:
- If `len` can be weakref’d: registration succeeds.
- Otherwise: registration raises `TypeError`.

Treat as informational rather than a strict contract unless you want to enforce one behavior.


In [29]:
p = SmartParameter(id=sp.Symbol('nw'))

try:
    p.register_callback(len)
    print("len was weakref-able here; ok.")
except TypeError:
    print("len not weakref-able; TypeError as expected.")


len not weakref-able; TypeError as expected.


---

# Part 1+2 Ends here

---

## Registry auto-vivify and convenience methods

Verifies `SmartParameterRegistry` behavior:
- `reg[symbol]` auto-creates a `SmartParameter` if missing.
- Same key returns the same instance (until overwritten/deleted).
- Convenience methods `set_value` and `set_protected` delegate to the parameter.
- Clamping and exclusion behavior still apply when called through the registry.


In [34]:
reg = SmartParameterRegistry()

a = sp.Symbol('a')
pa = reg[a]  # auto-create

assert isinstance(pa, SmartParameter)
assert pa.id == a
assert pa.value == 0

# same object returned for same key
assert reg[a] is pa

# set_value clamps by default bounds
pa.value=2.0
assert pa.value == 1.0

log = []
def cb(param, **kwargs):
    log.append(kwargs.get("owner_token"))

tok = pa.register_callback(cb)
pa.set_protected(-2.0, owner_token=tok)
assert log == []           # excluded
assert reg[a].value == -1.0


## Registry overwrite and deletion semantics

Verifies:
- Assigning `reg[x] = new_param` overwrites the stored instance for that key.
- Deleting `del reg[x]` removes it.
- Accessing again auto-vivifies a fresh parameter with default config.


In [35]:
reg = SmartParameterRegistry()
x = sp.Symbol('x')

p = reg[x]
assert reg[x] is p

p2 = SmartParameter(id=x, min_val=None, max_val=None, default_val=7)
reg[x] = p2
assert reg[x] is p2
assert reg[x].value == 7

del reg[x]
p3 = reg[x]  # auto-vivify fresh
assert p3 is not p2
assert p3.default_val == 0


## Multi-controller “no loop” pattern (logic-only)

Demonstrates (and tests) the intended controller pattern:
- Each controller registers a callback and receives a token.
- When controller updates the parameter, it uses `set_protected(..., owner_token=its_token)`
  so it **does not** get notified of its own update.
- Other controllers *do* update.

The last assertions intentionally allow “stale by design” behavior for the excluded view.


In [36]:

p = SmartParameter(id=sp.Symbol('A'))

view1 = {"value": None}
view2 = {"value": None}

def update_view1(param, **kwargs):
    view1["value"] = param.value

def update_view2(param, **kwargs):
    view2["value"] = param.value

t_view1 = p.register_callback(update_view1)
t_view2 = p.register_callback(update_view2)

def controller1_set(new_val):
    p.set_protected(new_val, owner_token=t_view1, source="controller1")

def controller2_set(new_val):
    p.set_protected(new_val, owner_token=t_view2, source="controller2")

controller1_set(0.8)
assert view1["value"] is None   # excluded
assert view2["value"] == 0.8

controller2_set(-10)
assert view1["value"] == -1.0
assert view2["value"] == 0.8    # excluded; may be stale by design
